# Guided Project: Answering Business Questions using SQL

## Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db
    

'Connected: None@chinook.db'

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM customer
limit 2

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5


In [4]:
%%sql
SELECT *
FROM employee
LIMIT 2

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


## Selecting Albums to Purchase

Next we find the top 3 genres by sales total from the USA

In [5]:
%%sql
WITH q1 AS
    (
    SELECT c.country, i.invoice_id, i.total
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
    ),
    q2 AS
    (
    SELECT il.invoice_id, g.name genre
    FROM invoice_line il
    INNER JOIN track t ON il.track_id = t.track_id
    INNER JOIN genre g ON t.genre_id = g.genre_id
    WHERE genre LIKE '%Hip%' OR genre LIKE '%Punk%' 
          OR genre LIKE '%Pop%' OR genre LIKE '%Blues%'
    )
SELECT q2.genre, sum(q1.total), q1.country
FROM q1
INNER JOIN q2 ON q1.invoice_id = q2.invoice_id

GROUP BY 1
ORDER BY 2 DESC




Done.


genre,sum(q1.total),country
Alternative & Punk,1234.5300000000018,USA
Blues,453.4199999999998,USA
Hip Hop/Rap,366.3,USA
Pop,201.95999999999992,USA


Based on the results of cash income, if we have to choose 3 of the four genres above we'd have to leave 'Pop' out.

Let's now see what would be the decision based on number of sold tracks.

In [6]:
%%sql
WITH q1 AS
    (
    SELECT c.country, i.invoice_id, i.total
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
    ),
    q2 AS
    (
    SELECT il.invoice_id, g.name genre
    FROM invoice_line il
    INNER JOIN track t ON il.track_id = t.track_id
    INNER JOIN genre g ON t.genre_id = g.genre_id
    WHERE genre LIKE '%Hip%' OR genre LIKE '%Punk%' 
          OR genre LIKE '%Pop%' OR genre LIKE '%Blues%'
    )
SELECT q2.genre, count(q1.total)



FROM q1
    
INNER JOIN q2 ON q1.invoice_id = q2.invoice_id

GROUP BY 1
ORDER BY 2 DESC



Done.


genre,count(q1.total)
Alternative & Punk,130
Blues,36
Pop,22
Hip Hop/Rap,20


The choice we made earlier that was based on income seems more adequate since the number of tracks sold is almost identical for 'Pop' and 'Hip-hop' genres

## Analyzing Employee Sales Performance

In [7]:
%%sql
WITH q1 AS
(
SELECT 
    c.support_rep_id empl_id,
    c.customer_id,
    sum(i.total) revenue
    
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY 2
ORDER BY 3 DESC
)
SELECT 
    e.first_name || ' ' || e.last_name empl_name,
    e.title,
    e.hire_date,
    count(q1.customer_id) num_clients,
    ROUND(sum(q1.revenue))
FROM employee e
LEFT JOIN q1 ON e.employee_id = q1.empl_id
GROUP BY 1
ORDER BY 5 DESC

Done.


empl_name,title,hire_date,num_clients,ROUND(sum(q1.revenue))
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,21,1732.0
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,20,1584.0
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,18,1394.0
Andrew Adams,General Manager,2016-08-14 00:00:00,0,None
Laura Callahan,IT Staff,2017-03-04 00:00:00,0,None
Michael Mitchell,IT Manager,2016-10-17 00:00:00,0,None
Nancy Edwards,Sales Manager,2016-05-01 00:00:00,0,None
Robert King,IT Staff,2017-01-02 00:00:00,0,None


The sales rep hired first has the highest sales numbers, while the one hired last has the lowest.

## Analyzing Sales by Country

In [8]:
%%sql
WITH q1 as /* counts clients per country*/
(
SELECT 
    c.country,
    count(c.customer_id) num_cust
from customer c
GROUP BY 1
ORDER BY 1 
),

q2 AS   /* sums sales per country */
(
SELECT
    c.country,
    sum(i.total) sales
FROM customer c
INNER JOIN invoice i ON c.customer_id= i.customer_id
GROUP BY 1
ORDER BY 1
),

q3 AS /* find sales per customer*/
(
SELECT 
    c.country,
    c.customer_id,
    sum(i.total) sales_cust
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY 2
ORDER BY 1
),

q4 AS  /*average value of sales per customer for each country*/
(
SELECT
    q3.country,
    AVG(q3.sales_cust) avg_cust_sales
FROM q3
GROUP BY 1
ORDER BY 1
),

q5 AS /*average invoice value per country*/
(
SELECT 
    c.country,
    avg(i.total) avg_invoice
FROM customer c
INNER JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY 1
ORDER BY 1
),

q6 AS
(
SELECT 
    CASE
        WHEN q1.num_cust = 1 THEN 'Other'
        ELSE q1.country
    END as country,
    sum(q1.num_cust) num_customers,
    round(sum(q2.sales)) sales,
    round(avg(q4.avg_cust_sales)) avg_cust_sales,
    round(avg(q5.avg_invoice)) avg_invoice
FROM q1
INNER JOIN q2 ON q1.country = q2.country
INNER JOIN q4 ON q4.country = q1.country
INNER JOIN q5 ON q5.country = q1.country
GROUP BY 1
ORDER BY 3 DESC
)

/* ====================================*/

SELECT *
FROM 
    (SELECT q6.*, CASE
                   WHEN q6.country = 'Other' THEN 1
                   ELSE 0
                 END ind
       FROM q6
    )
ORDER BY ind



Done.


country,num_customers,sales,avg_cust_sales,avg_invoice,ind
USA,13,1040.0,80.0,8.0,0
Canada,8,536.0,67.0,7.0,0
Brazil,5,428.0,86.0,7.0,0
France,5,389.0,78.0,8.0,0
Germany,4,335.0,84.0,8.0,0
Czech Republic,2,273.0,137.0,9.0,0
United Kingdom,3,246.0,82.0,9.0,0
Portugal,2,185.0,93.0,6.0,0
India,2,183.0,92.0,9.0,0
Other,15,1095.0,73.0,7.0,1


## Album vs Individual Tracks

In [9]:
%%sql
WITH 
q2 AS
(
SELECT t.album_id, il.track_id, il.invoice_id
FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
GROUP BY 3
)

SELECT 
    whole_album,
    count(distinct invoice_id) number_invoices, 
    cast(count(distinct invoice_id) as Float)/
                   (SELECT count(distinct invoice_id) FROM invoice) perc_inv
FROM 
    (
    SELECT q22.*, 
           CASE
            WHEN (SELECT t2.track_id /* whole album */
                  FROM track t2
                  WHERE t2.album_id = q22.album_id
                  
                  EXCEPT
                  
                  SELECT il2.track_id /* only purchased tracks */
                  FROM invoice_line il2
                  WHERE il2.invoice_id = q22.invoice_id
                 ) IS NULL
                    
                  AND
        
                  (
                  SELECT il2.track_id
                  FROM invoice_line il2
                  WHERE il2.invoice_id = q22.invoice_id
                  EXCEPT
                  SELECT t2.track_id
                  FROM track t2
                  WHERE t2.album_id = q22.album_id
                  
                  ) IS NULL
                  THEN 'Yes'
                  ELSE 'No'
              END as whole_album
    FROM q2 as q22
    )
GROUP BY whole_album
    

Done.


whole_album,number_invoices,perc_inv
No,500,0.8143322475570033
Yes,114,0.18566775244299674


 ## Next Steps

1. Which artist is used in the most playlists?
2. How many tracks have been purchased vs not purchased?
3. Is the range of tracks in the store reflective of their sales popularity?
4. Do protected vs non-protected media types have an effect on popularity?

#### *Which artist is used in the most playlists?*

In [10]:
%%sql
SELECT a.name artist, count(pl.name) in_playlists
FROM artist a
INNER JOIN album al ON a.artist_id = al.artist_id
INNER JOIN track t ON al.album_id = t.album_id
INNER JOIN playlist_track ptr ON t.track_id = ptr.track_id
INNER JOIN playlist pl ON ptr.playlist_id = pl.playlist_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 3

Done.


artist,in_playlists
Iron Maiden,516
U2,333
Metallica,296


#### *How many tracks have been purchased vs not purchased?*

In [11]:
%%sql
WITH q1 AS
(
SELECT *
FROM invoice_line
GROUP BY track_id
)

SELECT count(tracks), 
    CASE 
        WHEN purch IS NULL THEN 'No'
        ELSE 'Yes'
    END AS purchased
FROM 
    (
    SELECT t.track_id tracks, il.track_id purch
    FROM track t
    LEFT JOIN q1 il ON t.track_id = il.track_id
    )
GROUP BY 2

Done.


count(tracks),purchased
1697,No
1806,Yes


In [12]:
%%sql
SELECT count(distinct track_id)
FROM track

Done.


count(distinct track_id)
3503


#### *Is the range of tracks in the store reflective of their sales popularity?*

In [13]:
%%sql
WITH q1 AS
(
SELECT t.track_id, count(il.invoice_id) sales
FROM track t
LEFT JOIN invoice_line il ON t.track_id = il.track_id
GROUP BY 1
ORDER BY 2 DESC
)

SELECT CASE
        WHEN sales = 0 THEN '0. No Sales'
        WHEN sales >0 and sales<6 THEN '1. 1 to 5'
        WHEN sales >5 and sales<11 THEN '2. 6 to 10'
        WHEN sales >10 and sales<16 THEN '3. 11 to 15'
        ELSE '4. More than 15 sales'
        END AS sales_buckets,
        COUNT(sales) number_sales,
        round(100*CAST(COUNT(sales) as Float)/
                    (SELECT count(*) FROM q1),2) perc
FROM q1
GROUP BY 1
ORDER BY 1

Done.


sales_buckets,number_sales,perc
0. No Sales,1697,48.44
1. 1 to 5,1626,46.42
2. 6 to 10,165,4.71
3. 11 to 15,14,0.4
4. More than 15 sales,1,0.03


Most of the songs in the store have no sales at all. Only a handful (less than half a percent) have more than 10 sales.

#### *Do protected vs non-protected media types have an effect on popularity?*

In [14]:
%%sql
SELECT mt.name, count(il.invoice_id)
FROM track t
INNER JOIN media_type mt ON t.media_type_id = mt.media_type_id
LEFT JOIN invoice_line il ON t.track_id = il.track_id

GROUP BY 1

Done.


name,count(il.invoice_id)
AAC audio file,21
MPEG audio file,4259
Protected AAC audio file,439
Protected MPEG-4 video file,3
Purchased AAC audio file,35


In [23]:
%%sql
with q1 AS
(
SELECT track_id, count(track_id) num_tracks
FROM invoice_line
GROUP BY 1
ORDER BY 2 DESC
)

SELECT  CASE WHEN mt.name LIKE 'Protected%' THEN 'Yes'
                      ELSE 'No' END protected,
            sum(q1.num_tracks) num_tracks,
            CAST(sum(q1.num_tracks) AS Float)/(SELECT sum(q1.num_tracks)
            FROM q1) AS perc
FROM media_type mt
INNER JOIN track t ON mt.media_type_id = t.media_type_id
INNER JOIN q1 ON t.track_id = q1.track_id
GROUP BY 1

Done.


protected,num_tracks,perc
No,4315,0.9070842968257306
Yes,442,0.0929157031742695


Less than 10% of purchased tracks are 'Protected'